<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Классификация токсичных комментариев

Нужно построить инструмент, который будет искать токсичные комментарии.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# Импортирование необходимых модулей и атрибутов
import pandas as pd
import numpy as np
import os
import re
import nltk
import string
import lightgbm

from pymystem3 import Mystem
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score 
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

In [2]:
# Создадим вспомогательные переменные, константы
SEED = 12345

In [3]:
# Объявим функцию, которая будет читать файлы
def pth_load(pth1, pth2):
    """Sapport using os.path.exists. Load local file in primarily

    :param pth1: local addres of file
    :type pth1: object
    :param pth2: external addres of file
    :type pth2: object
    
    :raises ValueError: if file not found in addresses
    
    :rtype: DataFrame
    :return: foundly file in the form of DataFrame
    """
    if os.path.exists(pth1):
        df = pd.read_csv(pth1)
    elif os.path.exists(pth2):
        df = pd.read_csv(pth2)
    else:
        print('Something is wrong')
    return df

# Прочитаем файл, сохраним данные
df = pth_load('toxic_comments.csv', '/datasets/toxic_comments.csv')

In [4]:
# Вызовем метод 'info()' и напечатаем несколько строк. Прочитаем одну строку целиком
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
# Прочитаем одну строку целиком
df['text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [6]:
# Напишем функцию для очистки текста
def clear_text(text):
    """performs cleans the text

    :param text: text to clear
    :type text: object
    
    :rtype: object
    :return: cleared text
    """
    text = re.sub(r'[^a-z\' ^0-9]', ' ', text)
    #text = re.sub(r'[^a-z]', ' ', text)
    clean_text = " ".join(text.split())
    
    return clean_text

In [7]:
# Переведем символы в нижний регистр
df['lemm_text'] = df['text'].str.lower()

# Обрежем текст ячеек от лишних символов
df['lemm_text'] = df['lemm_text'].apply(lambda x: clear_text(x))

#df['lemm_text'] = df['lemm_text'].str.slice(0, 512) # можно обрезать ячейки до 512 символов в случае необходимости

Чтобы ускорить лемматизацию, объединим ячейки и вызовем функцию лемматизации к объединенной строке. Проблема в том, что многие функции лемматизации, например Mystem, при каждом вызове загружается заново, сильно замедляя процесс. Также можно применить распараллеливание процесса.

В данном случае мы используем WordNetLemmatizer, работающий с английским языком, но в рамках учебного процесса оставим этот метод.
Данный метод был описан на странице https://itnan.ru/post.php?c=1&p=503420

In [8]:
# Сколько ячеек объединяем в одну
batch_size = 1000

text_batch = [list(df['lemm_text'])[i: i + batch_size] for i in range(0, len(df), batch_size)]

In [9]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# Лемматизируем объединенные ячейки и разделим обратно на отдельные
def checkExecTimeMystemOneText(texts):
    """performs lemmatization the text. first it combines the text for joint processing, at the end it separates

    :param text: text to lemmatization
    :type text: object
    
    :rtype: object
    :return: lemmatized text
    """
    #m = Mystem()
    lemmatizer = WordNetLemmatizer()
    
    lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    txtpart = lol(texts, batch_size)
    res = []
    for txtp in txtpart:
        alltexts = ' '.join([txt + ' brblm ' for txt in txtp])
        
        #words = m.lemmatize(alltexts)
        words = [lemmatizer.lemmatize(w) for w in nltk.word_tokenize(alltexts)]
        
        doc = []
        for txt in words:
            if txt != '\n' and txt.strip() != '':
                if txt == 'brblm':
                    res.append(doc)
                    doc = []
                else:
                    doc.append(txt)
                    
        return res

processed_texts = Parallel(n_jobs=-1)(delayed(checkExecTimeMystemOneText)(t) for t in tqdm(text_batch))

100%|██████████| 160/160 [00:25<00:00,  6.29it/s]


In [11]:
# Получился вложенный список. Развернем верхний слой и объединим список в текст
lemm_text = sum(processed_texts, [])

# Удалим использованные переменные
del text_batch
del processed_texts

for i in range(0, len(lemm_text), 1):
    lemm_text[i] = " ".join(lemm_text[i]).replace(' \' ','\'')

# Сохраним в набор данных
df['lemm_text'] = lemm_text
del lemm_text

# Проверим результат
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159571 non-null  object
 1   toxic      159571 non-null  int64 
 2   lemm_text  159571 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i 'm see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i 'm really not trying to edit war it ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [12]:
# Соберем тренировочные и тестовые выборки, корпуса и таргеты
train, test = train_test_split(df, test_size=0.5, random_state=SEED)

corpus_train = train['lemm_text'].values.astype('U')
corpus_test = test['lemm_text'].values.astype('U')

target_train = train['toxic']
target_test = test['toxic']

nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Получим фичи на моделях. Для этого переведем текст в различные векторные выражения: мешок слов, N-граммы, TF-IDF.

In [13]:
# Создадим мешок слов
count_vect = CountVectorizer(dtype=np.float32, stop_words=stop_words)

bow_train = count_vect.fit_transform(corpus_train) 
print("Размер мешка:", bow_train.shape)

bow_test = count_vect.transform(corpus_test) 
print("Размер мешка:", bow_test.shape)

Размер мешка: (79785, 112899)
Размер мешка: (79786, 112899)


In [14]:
# Создадим N-граммы
ngramm_vect = CountVectorizer(dtype=np.float32, stop_words=stop_words, ngram_range=(2, 2))

n_gramm_train = ngramm_vect.fit_transform(corpus_train) 
print("Размер N-грамм:", n_gramm_train.shape)

n_gramm_test = ngramm_vect.transform(corpus_test) 
print("Размер N-грамм:", n_gramm_test.shape)

Размер N-грамм: (79785, 1483795)
Размер N-грамм: (79786, 1483795)


In [15]:
# Создадим TF-IDF 
count_tfidf = TfidfVectorizer(stop_words=stop_words)

tfidf_train = count_tfidf.fit_transform(corpus_train)
print("Размер матрицы:", tfidf_train.shape)

tfidf_test = count_tfidf.transform(corpus_test)
print("Размер матрицы:", tfidf_test.shape)

Размер матрицы: (79785, 112899)
Размер матрицы: (79786, 112899)


## Обучение

Изучим различные модели машинного обучения на разных фичах: мешке слов, N-граммах, TF-IDF.

In [16]:
# Для уменьшения кода используем функцию
def models_check(model, features_train, features_test, title):
    """trains the model and outputs the f1 metric

    :param model: machine learning model
    :type model: any machine learning model
    :param features_train: data set with train features
    :type features_train: csr_matrix or pd.DataFrame
    :param features_test: data set with test features
    :type features_test: csr_matrix or pd.DataFrame
    :param title: machine learning model
    :type title: description of models and features to display to the user
    
    
    :rtype: trained machine learning model
    :return: any machine learning model
    """
    model.fit(features_train, target_train)
    predictions = pd.Series(model.predict(features_test))
    
    print('Метрика F1 модели', title, '- {:.3f}'.format(f1_score(target_test, predictions)))
    
    return model, predictions

In [17]:
# Изучим дерево решений
tree_classifier = DecisionTreeClassifier(random_state=SEED,
                                         max_depth=13,
                                         min_samples_leaf=4)


# Вызовем дерево решений на мешке слов
tree_bow_model, tree_bow_predictions = models_check(tree_classifier, 
                                                    bow_train, 
                                                    bow_test, 
                                                    'DecisionTreeClassifier на мешке слов')

Метрика F1 модели DecisionTreeClassifier на мешке слов - 0.609


In [18]:
# Дерево решений на N_граммах
tree_n_gramm_model, tree_n_gramm_predictions = models_check(tree_classifier, 
                                                            n_gramm_train, 
                                                            n_gramm_test, 
                                                            'DecisionTreeClassifier на N_граммах')

Метрика F1 модели DecisionTreeClassifier на N_граммах - 0.174


In [19]:
# Дерево решений на TF-IDF
tree_tfidf_model, tree_tfidf_predictions = models_check(tree_classifier, 
                                                        tfidf_train, 
                                                        tfidf_test, 
                                                        'DecisionTreeClassifier на TF-IDF')

Метрика F1 модели DecisionTreeClassifier на TF-IDF - 0.615


In [20]:
# Изучим модель градиентного бустинга
lgbm_classifier = LGBMClassifier(random_state=SEED,
                                 max_depth=13,
                                 n_estimators=13,
                                 learning_rate=1.0)


# Вызовем модель градиентного бустинга на мешке слов
lgbm_bow_model, lgbm_bow_predictions = models_check(lgbm_classifier, 
                                                    bow_train, 
                                                    bow_test, 
                                                    'LGBMClassifier на мешке слов')

Метрика F1 модели LGBMClassifier на мешке слов - 0.642


In [21]:
# Модель градиентного бустинга на N_граммах
lgbm_n_gramm_model, lgbm_n_gramm_predictions = models_check(lgbm_classifier, 
                                                            n_gramm_train, 
                                                            n_gramm_test, 
                                                            'LGBMClassifier на N_граммах')

Метрика F1 модели LGBMClassifier на N_граммах - 0.240


In [22]:
# Модель градиентного бустинга на TF-IDF
lgbm_tfidf_model, lgbm_tfidf_predictions = models_check(lgbm_classifier, 
                                                        tfidf_train, 
                                                        tfidf_test, 
                                                        'LGBMClassifier на TF-IDF')

Метрика F1 модели LGBMClassifier на TF-IDF - 0.673


In [23]:
# Изучим модель логистической регрессии
log_reg_model = LogisticRegression(random_state=SEED, 
                                   max_iter=1000)


# Вызовем модель градиентного бустинга на мешке слов
log_reg_bow_model, log_reg_bow_predictions = models_check(log_reg_model, 
                                                          bow_train, 
                                                          bow_test, 
                                                          'LogisticRegression на мешке слов')

Метрика F1 модели LogisticRegression на мешке слов - 0.752


In [24]:
# Модель градиентного бустинга на N_граммах
log_reg_n_gramm_model, log_reg_n_gramm_predictions = models_check(log_reg_model, 
                                                          n_gramm_train, 
                                                          n_gramm_test, 
                                                          'LogisticRegression на N_граммах')

Метрика F1 модели LogisticRegression на N_граммах - 0.321


In [25]:
# Модель градиентного бустинга на TF-IDF
log_reg_tfidf_model, log_reg_tfidf_predictions = models_check(log_reg_model, 
                                                              tfidf_train, 
                                                              tfidf_test, 
                                                              'LogisticRegression на TF-IDF')

Метрика F1 модели LogisticRegression на TF-IDF - 0.710


## Выводы

Лучшую метрику F1 показала модель логистической регрессии, обучанная на мешке слов. Она же единственная, достигнувшая порога в 0.75.

Логистическая регрессия лучше всего обучилась на мешке слов и TF-IDF. Аналогично модель дерево решений, и градиентного бустинга. Но модель градиентного бустинга обучилась на TF-IDF лучше, чем на мешке слов. Все рассматриваемые модели на N_граммах показали худшие результаты.